# Failure Analysis CombinedNet

## Imports

### Packages

In [ ]:
import torch
import torch.nn as nn
from torch.utils import data
from torchvision import models
import pandas as pd
import numpy as np
import sys
import os
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, roc_curve, auc
import random

### Local

In [ ]:
sys.path.append(".")
sys.path.append("..")
from dataset.patient_dataset import PatientDataset
from models.lstm import LSTM
from models.omnipotent_resnet import Net
from models.combined_net import CombinedNet
from visualisation.slice_plotter import plot_patient_slices
from visualisation.analysis_plotter import plot_confusion_matrix, plot_roc_curve

## Parameters

In [ ]:
### DEFAULT PARAMETERS ###
### Data parameters ###
DATA_DIR = '../../../sliced_data/train'  # The slices we will train on for each patient
DS_DIR = '../../../data_split'
TARGET_SLICES = (0, 32)
### Model parameters ###
MODEL_DIR = '../models'  # Directory where best models are saved
TRAINED_MODEL = 'combinednet_006.pt'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Train on GPU or CPU
RESNET_MODEL_TYPE = 'resnet18'  # Which type of resnet is used by the model
### Test parameters ###
N_FEATURES = 128
BATCH_SIZE = 1

## Load Model

In [ ]:
# Load in correct model
if RESNET_MODEL_TYPE == "resnet50":
    model = models.resnet50()
elif RESNET_MODEL_TYPE == "resnet34":
    model = models.resnet34()
elif RESNET_MODEL_TYPE == "resnet18":
    model = models.resnet18()
else:
    print(f'No resnet with name {RESNET_MODEL_TYPE}')
    exit()
resnet = Net(model, 'combinednet', N_FEATURES)
lstm_net = LSTM(n_features=N_FEATURES, n_hidden=64, n_layers=2)
combined_net = CombinedNet(name='combinednet', cnn_net=resnet, lstm_net=lstm_net)
combined_net.load_state_dict(torch.load(os.path.join(MODEL_DIR, TRAINED_MODEL), map_location=DEVICE))
combined_net.to(DEVICE)
print("Loaded "+TRAINED_MODEL)

## Load Data

In [ ]:
val_df = pd.read_csv(os.path.join(DS_DIR, "val_df.csv"), names=["patient_nr", "slice_nr", "class"])
print(f"\nNumber of unique patient numbers in validation set: {len(np.unique(val_df['patient_nr']))}")
print(f"Number of unique slice numbers in validation set:   {len(np.unique(val_df['slice_nr']))}")
print(f"Number of unique class values in validation set:    {len(np.unique(val_df['class']))}")
    
val_patients = pd.read_csv(os.path.join(DS_DIR, "val_patients.csv"), names=["patient_nr"]).to_numpy().flatten()
print(f"Number of patient numbers in the validation patients list: {len(val_patients)}")

In [ ]:
print(val_df)

##  Dataloader

In [ ]:
analysis_set = PatientDataset(val_df, val_patients, TARGET_SLICES, DATA_DIR)
analysis_loader = data.DataLoader(analysis_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=os.cpu_count())

## Test Model

In [ ]:
combined_net.eval()
false_positives = []
false_negatives = []
probabilities = []
predictions = []
targets = []
    
for batch_idx, (image, target) in tqdm(enumerate(analysis_loader), total=len(analysis_loader), desc="Testing validation patients", leave=False):
    image = image.float().to(DEVICE)
    target = target.int().detach().cpu().data.numpy()[0]
    output = combined_net(image).detach().cpu()
    image = np.squeeze(image.detach().cpu().data.numpy())
    
    # Compute probabilities (requirement: round to 5 decimals)
    probability = np.round(torch.sigmoid(output).numpy(), 5)
    
    # Compute class from probability (>0.5 = abnormal)
    prediction = (probability > 0.5).astype(np.uint8)
    
    # Save probability, predicted and target values
    probabilities.append(probability)
    predictions.append(prediction)
    targets.append(target)
    
    # Save wrongly predicted images
    if(not bool(target) and bool(prediction)):
        false_positives.append(image)
    if(bool(target) and not bool(prediction)):
        false_negatives.append(image)

## Confusion Matrix

In [ ]:
plot_confusion_matrix(targets, predictions, title = "Patients Labeled by Network")

In [ ]:
tn, fp, fn, tp = confusion_matrix(targets, predictions).ravel()

In [ ]:
print("True Positives: "+str(tp))
print("True Negatives: "+str(tn))
print("False Positives: "+str(fp))
print("False Negatives: "+str(fn))

## Classification Metrics

In [ ]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Accuracy: {:.2f}".format(accuracy))
print("Sensitivity: {:.2f}".format(sensitivity))
print("Specificity: {:.2f}".format(specificity))

## ROC-curve & AUC 

In [ ]:
fpr, tpr, thresholds = roc_curve(targets, probabilities)
roc_auc = auc(fpr, tpr)
plot_roc_curve(fpr, tpr, roc_auc)

## Plot Images

### False Positive Patient
Normal patient labeled as an abnormal patient.

In [ ]:
fp_patient = random.choice(false_positives)
plot_patient_slices(fp_patient)

### False Negative Patient
Abnormal patient labeled as a normal patient.

In [ ]:
fn_patient = random.choice(false_negatives)
plot_patient_slices(fn_patient)